In [ ]:
import numpy as np
import pandas as pd
from plotly import graph_objects as go
import nevergrad as ng

from estival.wrappers.nevergrad import optimize_model
import estival.priors as esp
import estival.targets as est
from estival.model import BayesianCompartmentalModel
from estival.wrappers import pymc as epm
pd.options.plotting.backend = "plotly"
from summer2 import CompartmentalModel
from summer2.parameters import Parameter, DerivedOutput

from tb_incubator.demographics import add_extra_crude_birth_flow

In [ ]:
# Base model construction - also quite arbitrary
model_comps = ["susceptible", "early latent", "late latent", "infectious", "recovered"]
model_times = [1850.0, 2024.0]
model = CompartmentalModel(
    times=model_times,
    compartments=model_comps,
    infectious_compartments=["infectious"],
)
init_pops = {"susceptible": Parameter("starting population"), "infectious": 0.0}
model.set_initial_population(init_pops)
model.request_output_for_compartments("total_population", model_comps);

In [ ]:
# TB transitions, some meaningless TB-related flows
model.add_death_flow("TB death", Parameter("death rate"), "infectious")
model.add_transition_flow("silly_transition", Parameter("silly transition rate"), "susceptible", "infectious")

In [ ]:
# Demographic transitions
model.add_universal_death_flows("population_death", Parameter("population death rate"))
model.add_replacement_birth_flow("replacement_birth", "susceptible")
add_extra_crude_birth_flow(model, "extra_birth", Parameter("population growth rate"), "susceptible")

In [ ]:
params = {
    "silly transition rate": 1.0,
    "population growth rate": 0.001,
    "death rate": 0.1,
    "population death rate": 0.01,
    "starting population": 500.0,
}

In [ ]:
model.run(params)

In [ ]:
model.get_outputs_df().plot.area()

In [ ]:
outputs = model.get_derived_outputs_df()
outputs["check"] = params["starting population"] * np.exp(np.arange(0.0, model_times[1] - model_times[0] + 1, 1.0) * params["population growth rate"])
outputs.plot()

In [ ]:
# Prepare calibration model
priors = [
    esp.UniformPrior("population growth rate", (0.0, 0.2)),
    esp.UniformPrior("starting population", (0.0, 20000.0)),
]
target_pops = pd.Series({
    2010.0: 10000.0,
    2015.0: 11000.0,
})
targets = [est.NegativeBinomialTarget("total_population", target_pops, dispersion_param=100.0)]
bcm = BayesianCompartmentalModel(model, params, priors, targets)

In [ ]:
# Set up optimisation
budget = 1000
opt_class = ng.optimizers.NGOpt
orunner = optimize_model(bcm, opt_class=opt_class, budget=budget)
start_params = {"population growth rate": 0.1, "starting population": 500.0}
orunner = optimize_model(bcm, opt_class=opt_class, suggested=start_params, init_method="midpoint")

In [ ]:
rec = orunner.minimize(budget)
map_params = rec.value[1]
print("Best candidate parameters:")
for i_param, param in enumerate(map_params):
    print(f"   {param}: {round(map_params[param], 4)} (within bound {priors[i_param].bounds()}")

In [ ]:
model.run(parameters=params | map_params)

In [ ]:
outputs = model.get_derived_outputs_df()
fig = go.Figure()
fig.add_trace(go.Scatter(x=outputs.index, y=outputs["total_population"], name="modelled"))
fig.add_trace(go.Scatter(x=target_pops.index, y=target_pops, name="target", line={"width": 0.0}))
fig.update_layout(yaxis={"range": [0.0, 15000.0]})